## 네이버 쇼핑

In [1]:
from selenium import webdriver
# By: Tag에 접근하기 편하게 하는 기능
from selenium.webdriver.common.by import By
# Keys: 키보드의 이벤트
from selenium.webdriver.common.keys import Keys
import pandas as pd
from sqlalchemy import create_engine

In [4]:
# 웹드라이버를 이용하여 구글 크롬 브라우저 열기
driver = webdriver.Chrome()

In [5]:
# 특정 주소로 요청 보내기
driver.get('http://www.naver.com')

In [ ]:
# Tag 중 id가 query인 태그를 선택해 변수에 저장
search_element = driver.find_element(By.ID, 'query')

In [ ]:
type(search_element)
# WebElement = 웹에 있는 요소

selenium.webdriver.remote.webelement.WebElement

In [9]:
# search_element(검색어창)에 특정 텍스트를 입력(데이터 보내기)한다.
search_element.send_keys('아이폰')

In [11]:
# search_element에서 Enter 이벤트 발생시키기
search_element.send_keys(Keys.ENTER)

In [15]:
# 쇼핑 버튼(Tag) 선택하기
# = LINK_TEXT가 '쇼핑'인 Tag 찾기
# -> 해당 조건에 맞는 태그가 몇 개인가?
# find_element가 아닌 find_elements로 길이 확인 후, 여러 개면 다른 조건 사용해봐야 함.
# find_elements에서는 리스트의 함수만 사용 가능
len(driver.find_elements(By.LINK_TEXT, '쇼핑'))
# 조건에 맞는 태그의 개수가 1개이므로, 클릭 이벤트 발생
driver.find_element(By.LINK_TEXT, '쇼핑').click()

In [ ]:
# 쇼핑 페이지에서 검색어의 결과(물건의 가격과 제품명)를 크롤링
# -> 해당 페이지의 소스코드 불러오기
# 확인용
html_data = driver.page_source

In [19]:
# bs4 라이브러리 안에 있는 BeautifulSoup을 이용하여
# html_data를 parsing(데이터의 타입을 변환)
from bs4 import BeautifulSoup as bs

In [23]:
# 모은 정보가 첫 번째 탭의 정보일지, 두 번째 탭의 정보일지 알아보기
# html_data가 2개의 탭 중 어떤 소스코드인지 확인
# 탭의 제목이 다르기 때문에 title Tag 확인
# 특정 태그를 확인
soup.title

<title>아이폰 : 네이버 검색</title>

In [24]:
# page_source가 첫 번째 탭인 것을 확인
# 두 번째 탭으로 page_source를 이동
# 탭의 정보를 확인
# driver에서 두 탭의 주소 확인
driver.window_handles

['EA6F5AEFD4326D82916EAFC76E805A5B', 'C465B94DCECC7EF0B646E7F8D0E92833']

In [25]:
# driver에서 탭을 이동
driver.switch_to.window(
    driver.window_handles[1]
)

In [26]:
html_data2 = driver.page_source
soup2 = bs(html_data2, 'html.parser')

soup2.title

<title data-next-head="">아이폰 : 네이버 가격비교</title>

### 네이버 쇼핑 크롤링
1. div 태그 중 id가 'content'인 태그를 선택하여 저장 (content_data)
2. 저장한 태그(content data)에서 div 태그 중 class가 'product_item__KQayS'인 태그 정보를 모두 저장 (div_list)
3. div_list에서 div 태그 중 class가 'product_title__ljFM_'인 태그의 문자를 추출 (상품명)
4. div_list에서 span 태그 중 class가 'price'인 태그의 문자를 추출 (상품 가격)

In [ ]:
# 1.
content_data = soup2.find(
    'div',
    attrs={
        'id' : 'content'
    }
)
content_data

In [ ]:
# 2.
# content data에서 div 태그 중 class가 'product_item_KQayS'인 태그의 개수 확인
# find_all로 개수 찾고 find로 변경
len(
    content_data.find_all(
        'div',
        attrs={
            'class' : 'product_item__KQayS'
        }
    )
)

13

In [ ]:
# 또는
# product_item__KQayS, product_title__ljFM_ 등은 내일이 되면 바뀔 수도 있다.
# 따라서 class의 이름에 고정값인 'product_item'이 포함되어있는 태그를 찾는 방법도 있다.
# 다수의 데이터 중 내가 찾는 데이터가 있는지 확인하는 isin[] 사용
# => 똑같은 코드를 다른 때에 또 돌리게 될 때 훨씬 도움되는 방법
import re
len(
    content_data.find_all(
        'div',
        attrs={
            'class' : re.compile('product_item')
        }
    )
)

13

In [42]:
div_list = content_data.find_all(
            'div',
            attrs={
                'class' : 'product_item__KQayS'
            }
        )

#### for문 시작

In [ ]:
div_data = div_list[0]

In [53]:
item_name = div_data.find(
    'div',
    # attrs={
    #     'class' : 'product_title__ljFM_'
    # }
    attrs={
        'class' : re.compile('product_title')
    }
).get_text()

In [54]:
item_price = div_data.find(
    'span',
    attrs={
        'class':'price'
    }
).get_text()

In [56]:
# 상품의 이름과 상품의 가격을 dict 형태로 생성

# 비어있는 dict 생성
dict_data = {}

# dict에 데이터 추가
dict_data['상품명'] = item_name
dict_data['가격'] = item_price

dict_data

{'상품명': 'Apple 애플 아이폰 13 미니 128GB 새상품', '가격': '298,000원'}

#### for문 만들기

In [ ]:
values = []
for div_data in div_list:
    item_name = div_data.find('div', attrs={'class' : re.compile('product_title')}).get_text()
    item_price = div_data.find('span', attrs={'class':'price'}).get_text()
    item_url = div_data.find('a')['href']
    dict_data = {'상품명' : item_name, '가격' : item_price}
    values.append(dict_data)

In [59]:
values

[{'상품명': 'Apple 애플 아이폰 13 미니 128GB 새상품', '가격': '298,000원'},
 {'상품명': '아이폰 16 프로 128GB [자급제]', '가격': '최저1,538,900원'},
 {'상품명': '아이폰 16 프로 256GB [자급제]', '가격': '최저1,647,940원'},
 {'상품명': '애플 아이폰 13 미니 128GB 미개봉 새제품', '가격': '298,000원'},
 {'상품명': 'Apple 아이폰 12 mini 자급제  화이트  128GB', '가격': '458,000원'},
 {'상품명': '아이폰 16 128GB [자급제]', '가격': '최저1,232,000원'},
 {'상품명': '아이폰 16 256GB [자급제]', '가격': '최저1,394,000원'},
 {'상품명': '아이폰 15 프로 256GB [자급제]', '가격': '최저1,393,900원'},
 {'상품명': '아이폰 15 128GB [자급제]', '가격': '최저949,900원'},
 {'상품명': '해외아이폰 5S 무음 공기계 인스타 카메라 감성사진 학생폰 세컨폰 실버 16GB 항공', '가격': '79,500원'},
 {'상품명': '애플 아이폰17 프로 휴대폰성지 시세표 아이폰사전예약 공짜휴대폰 핸드폰성지 온라인개통', '가격': '100원'},
 {'상품명': '아이폰 16e 128GB [자급제]', '가격': '최저948,900원'},
 {'상품명': '아이폰 15프로 256GB Apple 미개봉', '가격': '574,000원'}]

In [ ]:
# div_list에서 div 태그 중 class의 이름에 'product_img_area'가 포함된 태그를 찾는다.
# 해당 태그에서 img 태그에 있는 src 속성의 값을 추출
div_data = div_list[0]

In [ ]:
# div_data에서 a태그(하이퍼링크) 첫 번째 정보를 확인하여 href 속성의 값을 출력
# html 문서에서 첫 번째 태그의 정보를 확인하는 방법 2가지
    # htmldata.태그명
    # htmldata.find(태그명)
# 특정 태그의 속성의 값을 확인하는 방법
    # 태그로 이루어진 데이터 Tag[속성명]
div_data.a['href']
div_data.find('a')['href']

'https://cr.shopping.naver.com/adcr?x=NSWFnSrZZDtKI9jg%2BTHCsP%2F%2F%2Fw%3D%3DsNovepaXkPL2svCP0k9bm561odJ2%2F6LBtogIwFjN3ri1yoYErd7Wx1dfdVO6hh7qVZ4TDxwAhNmm2sB7REec%2B%2BcKdlAuo4CRofAy%2FiHp8jqv8RtWXQ%2B4yWv%2FzzZujOpojmI2pf6w7%2BZPRPmsiCF%2FDqbnUM4PbNixF03EUqHcGaSHf0tRItGsVdjhlt3JkjekEyaxxRm4H8HcYDi0B3B3iQmfQT7uWwvxdz8nMvhXCKF1zPnBo0jGi%2FSdSlDWdPE2TPsT%2FxSjpUJFdlZfyHQtivweL8mDjug0MMBKQwp4NOEHBz7%2BORHa%2BX777%2F5U82WuFClV6w5UIcD7iElKPt4xFA%2FgE8e9QQc4K7zOyHVjcWSv6jQnn8m0skKQEOM3w8AiEmm1JLu3o30uX0kQP1eRkNfHIsJdUagYtylP20Ue0qWTXdwMZv%2BsjaOq%2FaIlKn6P2gbBqsN8Zes7AE1dCnTE379f04SLzEOLuWyApwkiBtsHdFr7skMvkOMBUq0V%2FWcMEKs1Ua8XULYTCnO9fYrVsTF7VsaTLuUmCaaMN%2FRmNua8phM4Xkqwer6KAYBzWkB5nEopvkYbWlTBuKOBID7cdbPWWjV8vjdCf6NvpU6ceBDKVwmwH%2B3aWprpLokeMIk%2FDaGimR4pR27mrh1IoDTOSgA%3D%3D&nvMid=88573763568&catId=50001519'

In [ ]:
# driver 스크롤을 가장 밑으로 내린다.
# driver.execute_script(
#     "window.scrollTo(0, document.body.scrollHeight);"
# )

In [ ]:
# driver에서 스크롤을 일정 간격으로 내린다.
driver.execute_script(
    "window.scrollBy(0, 800);"
)

In [90]:
html_data3 = driver.page_source

In [ ]:
# BeautifulSoup을 이용해서 데이터 파싱
soup3 = bs(html_data3, 'html.parser')
# 광고 상품을 제외한 모든 상품의 이름과 가격, 링크 주소를 2차원 데이터로 생성
content_data = soup3.find('div', attrs={'id':'content'})
div_list = content_data.find_all('div', attrs={'class': re.compile('product_item')})
# 특정 이름이 포함된 div 차기
values = []
for div_data in div_list:
    item_name = div_data.find('div', attrs={'class' : re.compile('product_title')}).get_text()
    item_price = div_data.find('span', attrs={'class':'price'}).get_text()
    item_url = div_data.find('a')['href']
    dict_data = {'상품명': item_name, '가격': item_price, 'url': item_url}
    values.append(dict_data)
# 생성된 2차원 데이터를 데이터프레임으로 생성
df = pd.DataFrame(values)
# 검색어('아이폰')를 파일명으로 csv 파일을 생성하고 인덱스는 제외한다.
df.to_csv('아이폰.csv', index=False)

In [ ]:
df

In [ ]:
# naver.shopping.py 파일에 코드 만들고 저장
# 그 파일 들어있는 폴더 경로 복사
# cmd (관리자 권한) 열고 'cd 복사한경로' 입력 => cd: change directory
# python naver.shopping.py 입력
# => 자동으로 검색하고 스크롤 내리고 csv 생성되고 종료됨

# 크롬에서 돌릴 때 너무 느리다면 pip install webdriver_manager